In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_txt(dfa, nome):
    dfa.to_csv('%s.txt' %nome, sep='\t', index=False, header=False)

# 1. Ler e Mudar nomes
- Ficheiros base

In [2]:
%%time
dfDelta=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\1Delta-Sonae BQ.xlsx", sheet_name="Sheet1")
#dfDelta2=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\Stocks Sonae_Liquida.xlsx", sheet_name="Sheet1")
#dfDelta=pd.concat([dfDelta1, dfDelta2], ignore_index=True)

dfNinjas1=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\2Ninja-CafesCervejaSonae.xlsx")
dfNinjas2=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\2Ninja-CafesSonae.xlsx")
dfNinjas3=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\2Ninja-CervejaSonae.xlsx")
dfNinjasO=pd.concat([dfNinjas1, dfNinjas2, dfNinjas3], ignore_index=True, axis=0, sort=False)

dfSonae1=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\3Sellout-CafeSonae3d.xlsx", sheet_name="Lista Lojas Sonae")
dfSonae2=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\3Sellout-CervSonae3d.xlsx", sheet_name="Lista Lojas Sonae")
dfSonae3=pd.merge(dfSonae1, dfSonae2, how="outer", on = "STORE")#,"Descrição Loja"])

dfNinjasO = dfNinjasO.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE"})

dfSonae3['Vendedor_x'] = dfSonae3['Vendedor_x'].fillna(dfSonae3['Vendedor_y'])
dfSonae3['STORE_NAME']=dfSonae3['STORE_NAME'].fillna(dfSonae3['Descrição Loja'])
dfSonae3=dfSonae3.rename(columns={"Vendedor_x": "Vendedor"})
dfSonae3=dfSonae3.drop(columns=['Descrição Loja', 'Vendedor_y'])
dfSonae3['STORE_NAME'] = dfSonae3['STORE_NAME'].str.upper()                                  

Wall time: 12.9 s


- Ficheiro reposição

In [3]:
dfSonae4=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\3Reposição.xlsx")

# Meter no ficheiro 3
dfSonae3['Reposição'] = ["Sim" if val in dfSonae4['Descrição Loja'].values else "Não" for val in dfSonae3['STORE_NAME']]

- Estabelecer variáveis

In [4]:
dfNinjas=dfNinjasO.copy()
produtos=dfNinjasO.columns[5:10]
presença=dfNinjasO.columns[4:10]

- Criar coluna de número de presenças e percentagem do expectável

In [5]:
dfNinjas['Presença'] = dfNinjas[presença].sum(axis=1, skipna=True)
dfNinjas['Presença_em_Percentagem'] = (dfNinjas['Presença'] / dfNinjas[presença].notna().sum(axis=1)) * 100

In [6]:
dfNinjas.head()

,Hora,DATA,Nome da Loja,STORE,Promoção,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ DELTA Q AQTIVUS 10CAP,CAFÉ DELTA Q DEQAFEINATUS 10CAP,"CERV.C/ALC.T/P CORONA 35,5CL",CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL,Presença,Presença_em_Percentagem
0,17:07:13,2023-04-01,Continente Arrábida (Gaia Jardim),460,1.0,1.0,1.0,1.0,1.0,1.0,6.0,100.000000
1,19:54:22,2023-04-02,Continente Aveiro,458,1.0,1.0,1.0,1.0,1.0,1.0,6.0,100.000000
2,18:27:18,2023-03-31,Continente Barreiro,927,0.0,1.0,1.0,1.0,1.0,1.0,5.0,83.333333
3,10:51:34,2023-03-31,Continente Beja,207,1.0,1.0,1.0,1.0,1.0,1.0,6.0,100.000000
4,21:49:00,2023-04-02,Continente Braga,459,1.0,1.0,1.0,1.0,1.0,1.0,6.0,100.000000


# Como o sellout foi dados para o fds, dividir por 3 para fazer por dia

In [7]:
dfSonae=dfSonae3.copy()
dividir=dfSonae3.columns[3:-1]
dfSonae[dividir]=dfSonae3[dividir]/3
dfSonae = dfSonae.rename(columns={'DELTA Q AQTIVUS CÁP. CX 10 UN': 'SELLOUT CAFÉ DELTA Q AQTIVUS 10CAP', 
                                  "DELTA Q DEQAFEINATUS CÁP. CX 10 UN":"SELLOUT CAFÉ DELTA Q DEQAFEINATUS 10CAP",
                                 "DELTA Q-QALIDUS CÁP. CX 10 UN":"SELLOUT CAFÉ DELTA Q QALIDUS 10CAP",
                                 "Cerveja Franziskaner Weiss 50cl (S/N)":"SELLOUT CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL",
                                 "Cerveja Corona 35,5 CL(S/N)":"SELLOUT CERV.C/ALC.T/P CORONA 35,5CL"})
dfSonae=dfSonae.iloc[:, [0,1,2,5,3,4,7,6,8]]

## O que tem cada ficheiro:

### **dfDelta**:
1. Formato long (produtos numa única coluna "DESC_ARTIGO")
2. Stocks, em Trânsito, Esperado, Linear, sellout dia anterior	


### **dfNinjas**:
1. Formato wide (cada produto numa coluna)
2. Presença de produto em linear para cada produto

### **dfSonae**:
1. Formato wide
2. Sellouts, Vendedores, Reposição


# VER

In [8]:
#dfDelta.info()
#dfNinjas.head()
#dfSonae.info()

# 2. Mudar para datetime os que forem precisos

In [9]:
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format=('%d-%m-%Y'))
#dfSonae['DATA'] = pd.to_datetime(dfSonae['DATA'])

# 3. Pedir dados fundamentais e mergir

In [10]:
#x = int(input("Número de colunas que não são produto em dfNinjasO: "))
#y = int(input("A partir de qual coluna aparece produto: "))
x=5
y=6
n = len(dfNinjasO.columns)-x                                     #Para automatizar, 6 colunas sem produtos

nomesProd = dfNinjasO.columns[y-1:y-1+n]
nomesResto = dfNinjasO.columns.difference(nomesProd)

> ## Código para Delta (Stocks, mergir, só mantemos colunas de interesse)

In [11]:
dfMeio=dfNinjas.copy()                                      #Dados Delta

for i in range (1,int(n)+1):                                #Quantos dados
    #coluna=input("Nome da coluna %s " % str(i))            #Nome do café
    coluna=dfNinjas.columns[int(y)+i-2]                     #Para automatizar os produtos aparecem a partir da 5ª coluna
    
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "SOH", "PRES_STOCK", "INTRANSIT", "EXPECTED"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["DATA","STORE"])            #Juntar dados de acordo com o dia e a loja
    dfMeio=dfMeio.rename(columns={"SOH": "STOCK %s" % coluna, "PRES_STOCK":"PRES_STOCK %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna})  #Nomear coluna nova    
    
    


In [12]:
#escrever_excel(dfMeio, "Ninja+Delta")

> ## Código para Sonae (Sellouts, mergir)

In [13]:
dfCompleto= pd.merge(dfMeio, dfSonae, how="left", on = ["STORE"])
dfCompleto=dfCompleto.drop(columns="STORE_NAME")

> # Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

> ## Fazer:
- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock
- Dias para a rotura

In [14]:
# Estabelecer qual o dataframe em questão

dfFinal=dfCompleto.copy()

> - Rotura

In [15]:
%%time

for i in nomesProd:
    dfFinal["Rotura %s" %i] = (dfFinal["STOCK %s" %i] == 0).astype(int) #Mais rápido

Wall time: 0 ns


> - Sinal

In [16]:
for i in nomesProd:
    sinal = "Sinal %s" % i
    ninja = "%s" % i
    stock = "STOCK %s" % i
    
    dfFinal[sinal] = np.where(dfFinal[ninja].astype(bool) & dfFinal[stock].astype(bool), "PRODUTO PRESENTE - COM STOCK",
                         np.where(~dfFinal[ninja].astype(bool) & dfFinal[stock].astype(bool), "PRODUTO AUSENTE - COM STOCK",
                                  np.where(dfFinal[ninja].astype(bool) & ~dfFinal[stock].astype(bool), "PRODUTO PRESENTE - SEM STOCK", 
                                           "PRODUTO AUSENTE - SEM STOCK")))

> - Ciclos

In [17]:
for i in nomesProd:
    dfFinal["Ciclos %s" %i] = dfFinal["STOCK %s" %i]/dfFinal["PRES_STOCK %s" %i]

> - Adequação

In [18]:
for i in nomesProd:
    adequa = "Adequação de %s" % i
    ciclos = "Ciclos %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRES_STOCK %s" % i
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, "Stock Suficiente", 
                               np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito]+dfFinal[esperado]+dfFinal[stock]>=dfFinal[linear]), "Stock Insuf c Forn Adequado", 
                                np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito]+dfFinal[esperado]+dfFinal[stock]<dfFinal[linear]), "Stock Insuf c Forn Desadequado", "")))

> - Dias para rotura

In [19]:
for i in nomesProd:
    dfFinal["Dias para Rotura %s" %i]=dfFinal["STOCK %s" %i]/dfFinal["SELLOUT %s" %i]

# Reomear e organizar

In [20]:
dfFinal=dfFinal.rename(columns={"VND (D-1)": "Sellout_1_Dia_Antes", "Nome da Loja": "Loja" })


# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura

#Desvio padrão

# Escrever

- Base de dados

In [316]:
#escrever_excel(dfFinal, "zFinalWide2")

- Ficheiro de produtos

In [179]:
dfProdutos = pd.DataFrame(produtos, columns=['produtos'])
dfProdutos.loc[-1] = [len(produtos)]
dfProdutos.index = dfProdutos.index + 1
dfProdutos = dfProdutos.sort_index() 

In [180]:
dfProdutos

,produtos
0,5
1,CAFÉ DELTA Q QALIDUS 10CAP
2,CAFÉ DELTA Q AQTIVUS 10CAP
3,CAFÉ DELTA Q DEQAFEINATUS 10CAP
4,"CERV.C/ALC.T/P CORONA 35,5CL"
5,CERV.C/ALC.T/P FRANZISKANER WEISSBI.50CL


In [181]:
#escrever_txt(dfProdutos, "zProdutos")